In [2]:
import pandas as pd 
import os

In [3]:
# CSV for states fullname
states_fullname = pd.read_csv('statelatlong.csv').rename(columns={'State':'State Code','City': 'State'})
states_fullname.head()

,State Code,Latitude,Longitude,State
0,AL,32.601011,-86.680736,Alabama
1,AK,61.302501,-158.775020,Alaska
2,AZ,34.168219,-111.930907,Arizona
3,AR,34.751928,-92.131378,Arkansas
4,CA,37.271875,-119.270415,California


In [4]:
file_list = os.listdir("Data")

In [5]:
list_of_dataframes = []
for filename in file_list:
    list_of_dataframes.append(pd.read_table("Data/"+filename))

merged_df = pd.concat(list_of_dataframes)


In [6]:
# adding states full names to full dataset
merged_df = pd.merge(merged_df,states_fullname, on='State Code', how='outer')

In [7]:
merged_df.head()

,Analyte ID,Analyte Name,State Code,PWSID,System Name,System Type,Retail Population Served,Adjusted Total Population Served,Source Water Type,Water Facility ID,...,Detection Limit Code,Detect,Value,Unit,Presence Indicator Code,Residual Field Free Chlorine mg/L,Residual Field Total Chlorine mg/L,Latitude,Longitude,State
0,1005.0,ARSENIC,AL,AL0000943,GUNTERSVILLE WATER WORKS & SEWER BOARD,C,12750.0,13980.0,SW,1553.0,...,MDL,0.0,NaN,NaN,NaN,NaN,NaN,32.601011,-86.680736,Alabama
1,1005.0,ARSENIC,AL,AL0000091,BRENT UTILITIES BOARD,C,5850.0,6885.0,GW,4221.0,...,MDL,0.0,NaN,NaN,NaN,NaN,NaN,32.601011,-86.680736,Alabama
2,1005.0,ARSENIC,AL,AL0000933,ALBERTVILLE UTILITIES BOARD,C,30186.0,45915.0,SW,1665.0,...,MDL,0.0,NaN,NaN,NaN,NaN,NaN,32.601011,-86.680736,Alabama
3,1005.0,ARSENIC,AL,AL0000398,CULLMAN UTILITIES BOARD,C,32.0,31112.0,SW,1346.0,...,MDL,0.0,NaN,NaN,NaN,NaN,NaN,32.601011,-86.680736,Alabama
4,1005.0,ARSENIC,AL,AL0001780,FIVE STAR WATER SUPPLY DISTRICT,C,100.0,6040.0,SW,4793.0,...,MDL,0.0,NaN,NaN,NaN,NaN,NaN,32.601011,-86.680736,Alabama


In [13]:
# remove repeated values skewing dating
no_repeat_locations=merged_df.drop_duplicates(subset=['Analyte Name', 'System Name'])
# Shorten column list
clean_columns = no_repeat_locations[['Analyte ID', 'Analyte Name', 'State Code','State','Retail Population Served','Source Water Type','Sample Collection Date','Detection Limit Value','Detection Limit Unit','Detect', 'Value','Unit']]
# change units to uppercase and strip extra spaces to unify
clean_columns['Unit'] = clean_columns['Unit'].str.upper().str.rstrip()


In [28]:
# unify all measurement for each contaminant if possible AND DROP FECAL COLIFORM
clean_columns.drop(clean_columns.index[clean_columns['Analyte Name'] == 'FECAL COLIFORM'], inplace = True)
clean_columns.loc[clean_columns['Analyte Name']=='ARSENIC', 'Unit'] ='MG/L'
clean_columns.loc[clean_columns['Analyte Name']=='ASBESTOS', 'Unit'] ='MFL'
clean_columns.loc[clean_columns['Analyte Name']=='CYANIDE', 'Unit'] ='MG/L'
clean_columns.loc[clean_columns['Analyte Name']=='MERCURY', 'Unit'] ='MG/L'
clean_columns.loc[clean_columns['Analyte Name']=='OXAMYL', 'Unit'] ='UG/L'
clean_columns.loc[clean_columns['Analyte Name']=='PICLORAM', 'Unit'] ='UG/L'
clean_columns.loc[clean_columns['Analyte Name']=='SELENIUM', 'Unit'] ='MG/L'
clean_columns.loc[clean_columns['Analyte Name']=='TOTAL POLYCHLORINATED BIPHENYLS (PCB)', 'Unit'] ='UG/L'

In [29]:
# unify all measurement for each contaminant if possible
clean_columns.groupby(['Analyte Name'])['Unit'].unique()


Analyte Name
ARSENIC                                  [MG/L]
ASBESTOS                                  [MFL]
CYANIDE                                  [MG/L]
FLUORIDE                                 [MG/L]
MERCURY                                  [MG/L]
OXAMYL                                   [UG/L]
PICLORAM                                 [UG/L]
SELENIUM                                 [MG/L]
TOTAL POLYCHLORINATED BIPHENYLS (PCB)    [UG/L]
Name: Unit, dtype: object

In [35]:
# Change null Values to 0 
clean_columns['Value'].fillna(0)

0          0.000
1          0.000
2          0.000
3          0.000
4          0.000
           ...  
1615962    0.000
1615963    0.000
1615964    0.001
1615967    0.420
1615968    0.000
Name: Value, Length: 332815, dtype: float64